In [37]:
import pandas as pd 
import math
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
vancover_data = pd.read_csv("./vancouver-parent_data.csv") 
# Preview the first 5 lines of the loaded data 
vancover_data.head(5)
print(vancover_data.shape[0])

193378


In [38]:
# Drop uneeded columns from dataset
new_van_data = vancover_data.drop(['TYPE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'], axis=1)
new_van_data.head(5)

,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
0,10XX SITKA SQ,Fairview,490612.964805,5.457110e+06
1,10XX ALBERNI ST,West End,491007.779775,5.459174e+06
2,10XX ALBERNI ST,West End,491051.085574,5.459144e+06
3,10XX ALBERNI ST,West End,491058.816893,5.459123e+06
4,10XX ALBERNI ST,West End,491067.645985,5.459114e+06


In [39]:
# This is a function that converts a x,y into long and lat values and then returns it

def utmToLatLng(zone, easting, northing, northernHemisphere=True):
    if not northernHemisphere:
        northing = 10000000 - northing

    a = 6378137
    e = 0.081819191
    e1sq = 0.006739497
    k0 = 0.9996

    arc = northing / k0
    mu = arc / (a * (1 - math.pow(e, 2) / 4.0 - 3 * math.pow(e, 4) / 64.0 - 5 * math.pow(e, 6) / 256.0))

    ei = (1 - math.pow((1 - e * e), (1 / 2.0))) / (1 + math.pow((1 - e * e), (1 / 2.0)))

    ca = 3 * ei / 2 - 27 * math.pow(ei, 3) / 32.0

    cb = 21 * math.pow(ei, 2) / 16 - 55 * math.pow(ei, 4) / 32
    cc = 151 * math.pow(ei, 3) / 96
    cd = 1097 * math.pow(ei, 4) / 512
    phi1 = mu + ca * math.sin(2 * mu) + cb * math.sin(4 * mu) + cc * math.sin(6 * mu) + cd * math.sin(8 * mu)

    n0 = a / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (1 / 2.0))

    r0 = a * (1 - e * e) / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (3 / 2.0))
    fact1 = n0 * math.tan(phi1) / r0

    _a1 = 500000 - easting
    dd0 = _a1 / (n0 * k0)
    fact2 = dd0 * dd0 / 2

    t0 = math.pow(math.tan(phi1), 2)
    Q0 = e1sq * math.pow(math.cos(phi1), 2)
    fact3 = (5 + 3 * t0 + 10 * Q0 - 4 * Q0 * Q0 - 9 * e1sq) * math.pow(dd0, 4) / 24

    fact4 = (61 + 90 * t0 + 298 * Q0 + 45 * t0 * t0 - 252 * e1sq - 3 * Q0 * Q0) * math.pow(dd0, 6) / 720

    lof1 = _a1 / (n0 * k0)
    lof2 = (1 + 2 * t0 + Q0) * math.pow(dd0, 3) / 6.0
    lof3 = (5 - 2 * Q0 + 28 * t0 - 3 * math.pow(Q0, 2) + 8 * e1sq + 24 * math.pow(t0, 2)) * math.pow(dd0, 5) / 120
    _a2 = (lof1 - lof2 + lof3) / math.cos(phi1)
    _a3 = _a2 * 180 / math.pi

    latitude = 180 * (phi1 - fact1 * (fact2 + fact3 + fact4)) / math.pi

    if not northernHemisphere:
        latitude = -latitude

    longitude = ((zone > 0) and (6 * zone - 183.0) or 3.0) - _a3

    return (latitude, longitude)

In [40]:
# Number of rows in the dataset
print(new_van_data.shape[0])

193378


In [41]:
# Here we are sending all our x,y to get a list of corresponding lat and long
latitudes = []
longitudes = []
#longitude is the first result and latitude is the second
for i in range(0, len(new_van_data['Y'].values), 1000):
    latits = new_van_data['X'].values[i:i+1000]
    longits = new_van_data['Y'].values[i:i+1000]
    for x, y in zip(latits, longits):
        result = utmToLatLng(10, x, y)
        latitudes.append(result[0])
        longitudes.append(result[1])    
print(len(longitudes))

193378


In [42]:
# We set our long and lat values in our dataset
new_van_data['longitude'] = longitudes
new_van_data['latitude'] = latitudes
new_van_data.head(5)

,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,longitude,latitude
0,10XX SITKA SQ,Fairview,490612.964805,5.457110e+06,-123.129029,49.266678
1,10XX ALBERNI ST,West End,491007.779775,5.459174e+06,-123.123649,49.285255
2,10XX ALBERNI ST,West End,491051.085574,5.459144e+06,-123.123053,49.284981
3,10XX ALBERNI ST,West End,491058.816893,5.459123e+06,-123.122946,49.284794
4,10XX ALBERNI ST,West End,491067.645985,5.459114e+06,-123.122824,49.284715


In [43]:
cols = new_van_data.columns.tolist()
print(cols)

['HUNDRED_BLOCK', 'NEIGHBOURHOOD', 'X', 'Y', 'longitude', 'latitude']


In [44]:
# We get rid of X,Y columns
new_cols = [cols[0], cols[1], cols[4], cols[5]]
new_data = new_van_data[new_cols]
new_data.head(5)

,HUNDRED_BLOCK,NEIGHBOURHOOD,longitude,latitude
0,10XX SITKA SQ,Fairview,-123.129029,49.266678
1,10XX ALBERNI ST,West End,-123.123649,49.285255
2,10XX ALBERNI ST,West End,-123.123053,49.284981
3,10XX ALBERNI ST,West End,-123.122946,49.284794
4,10XX ALBERNI ST,West End,-123.122824,49.284715


In [45]:
# Renaming the columns here
data = new_data.rename(columns={"HUNDRED_BLOCK": "location_name", "NEIGHBOURHOOD": "neighborhood"}, errors="raise")
data.head(5)

,location_name,neighborhood,longitude,latitude
0,10XX SITKA SQ,Fairview,-123.129029,49.266678
1,10XX ALBERNI ST,West End,-123.123649,49.285255
2,10XX ALBERNI ST,West End,-123.123053,49.284981
3,10XX ALBERNI ST,West End,-123.122946,49.284794
4,10XX ALBERNI ST,West End,-123.122824,49.284715


In [46]:
print('Total number of rows, including duplicates:', data.shape[0])

Total number of rows, including duplicates: 193378


In [47]:
# Dropping our duplicate location_name rows here
data.drop_duplicates(subset=['location_name'], inplace = True)

#WE NO LONGER WANT TO DROP DUPLICATES
print('Total number of rows, without duplicates:',data.shape[0])

Total number of rows, without duplicates: 16170


In [48]:
data.head(5)

,location_name,neighborhood,longitude,latitude
0,10XX SITKA SQ,Fairview,-123.129029,49.266678
1,10XX ALBERNI ST,West End,-123.123649,49.285255
10,10XX BARCLAY ST,West End,-123.126206,49.282661
18,10XX BEACH AVE,West End,-123.134768,49.276833
50,10XX BOUNDARY RD,Hastings-Sunrise,-123.023707,49.275879


In [49]:
# Here we are adding vancouver to city for the entire dataset
data['city'] = ['Vancouver' for i in range(data.shape[0])]
data.head(5)

,location_name,neighborhood,longitude,latitude,city
0,10XX SITKA SQ,Fairview,-123.129029,49.266678,Vancouver
1,10XX ALBERNI ST,West End,-123.123649,49.285255,Vancouver
10,10XX BARCLAY ST,West End,-123.126206,49.282661,Vancouver
18,10XX BEACH AVE,West End,-123.134768,49.276833,Vancouver
50,10XX BOUNDARY RD,Hastings-Sunrise,-123.023707,49.275879,Vancouver


In [50]:
# Create a unique id per row in the database for location_key
import uuid
location_key = []
for i in range(data.shape[0]):
    id = uuid.uuid4() 
    location_key.append(id)
print(len(location_key))

16170


In [51]:
# Add location_key into dataframe
data['location_key'] = location_key
data.head(5)

,location_name,neighborhood,longitude,latitude,city,location_key
0,10XX SITKA SQ,Fairview,-123.129029,49.266678,Vancouver,2796697f-010a-4467-a472-41bd6d18f995
1,10XX ALBERNI ST,West End,-123.123649,49.285255,Vancouver,dacda9c3-03f7-41fb-836c-e82fbc7aee0b
10,10XX BARCLAY ST,West End,-123.126206,49.282661,Vancouver,330c1261-474e-4aa3-9d1c-e0c2f0271738
18,10XX BEACH AVE,West End,-123.134768,49.276833,Vancouver,34c1355d-3fbf-492e-a8ac-863fff855b96
50,10XX BOUNDARY RD,Hastings-Sunrise,-123.023707,49.275879,Vancouver,f1ff9332-be7c-4676-9bb9-bbc900b990fe


In [52]:
cols = data.columns.tolist()
print(cols)

['location_name', 'neighborhood', 'longitude', 'latitude', 'city', 'location_key']


In [53]:
# Here we shift the order of the columns so it goes id, name, neighborhood, city, long, lat - to match sql table.
new_cols = [cols[5], cols[0], cols[1], cols[4], cols[2], cols[3]]
data = data[new_cols]
data.head(5)
print('Total number of rows, with duplicates:',data.shape[0])

Total number of rows, with duplicates: 16170


In [54]:
# Finally here we convert the dataframe to a csv file to store in our repo
vancouver_address_csv = data.to_csv(r'./vancouver-address.csv', index = None, header=True)